<a href="https://colab.research.google.com/github/nithin-k-mundrathi/LLM-practice/blob/main/RAG/RAG_C_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Retriver

In [1]:
import subprocess

url = "https://raw.githubusercontent.com/Denis2054/RAG-Driven-Generative-AI/main/commons/grequests.py"
output_file = "grequests.py"

# Prepare the curl command
curl_command = [
    "curl",
    "-o", output_file,
    url
]

# Execute the curl command
try:
    subprocess.run(curl_command, check=True)
    print("Download successful.")
except subprocess.CalledProcessError:
    print("Failed to download the file.")

Download successful.


In [2]:
!pip install requests==2.32.3

In [3]:

!pip install beautifulsoup4==4.12.3

## Fecth the Data

In [4]:
import requests
from bs4 import BeautifulSoup
import re

# URLs of the Wikipedia articles mapped to keywords
urls = {
    "prompt engineering": "https://en.wikipedia.org/wiki/Prompt_engineering",
    "artificial intelligence":"https://en.wikipedia.org/wiki/Artificial_intelligence",
    "llm": "https://en.wikipedia.org/wiki/Large_language_model",
    "llms": "https://en.wikipedia.org/wiki/Large_language_model"
}

## Process the Data

In [5]:
def fetch_and_clean(url):
    # Fetch the content of the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the main content of the article, ignoring side boxes and headers
    content = soup.find('div', {'class': 'mw-parser-output'})

    # Remove less relevant sections such as "See also", "References", etc.
    for section_title in ['References', 'Bibliography', 'External links', 'See also']:
        section = content.find('span', {'id': section_title})
        if section:
            for sib in section.parent.find_next_siblings():
                sib.decompose()
            section.parent.decompose()

    # Focus on extracting and cleaning text from paragraph tags only
    paragraphs = content.find_all('p')
    cleaned_text = ' '.join(paragraph.get_text(separator=' ', strip=True) for paragraph in paragraphs)
    cleaned_text = re.sub(r'\d+', '', cleaned_text)  # Remove citation markers like [1], [2], etc.

    return cleaned_text

# retieval Process for User Input

In [6]:
import textwrap

def process_query(user_input, num_words):
    user_input = user_input.lower()

    # Check for any of the specified keywords in the input
    matched_keyword = next((keyword for keyword in urls if keyword in user_input), None)

    if matched_keyword:
        print(f"Fetching data from: {urls[matched_keyword]}")
        cleaned_text = fetch_and_clean(urls[matched_keyword])

        # Limit the display to the specified number of words from the cleaned text
        words = cleaned_text.split()  # Split the text into words
        first_n_words = ' '.join(words[:num_words])  # Join the first n words into a single string

        # Wrap the first n words to 80 characters wide for display
        wrapped_text = textwrap.fill(first_n_words, width=80)
        print("\nFirst {} words of the cleaned text:".format(num_words))
        print(wrapped_text)  # Print the first n words as a well-formatted paragraph

        # Use the exact same first_n_words for the GPT-4 prompt to ensure consistency
        prompt = f"Summarize the following information about {matched_keyword}:\n{first_n_words}"
        wrapped_prompt = textwrap.fill(prompt, width=80)  # Wrap prompt text
        print("\nPrompt for Generator:", wrapped_prompt)

        # Return the specified number of words
        return first_n_words
    else:
        print("No relevant keywords found. Please enter a query related to 'LLM', 'LLMs', or 'Prompt Engineering'.")
        return None

# generator

### 2.1.Adaptive RAG Selection Based on Human Ranking: Integrating HF-RAG for Augmented Document Inputs

In [7]:
#@title dowload image
from grequests import download

# Define your variables
directory = "/content/RAG-Driven-Generative-AI/Chapter05"
filename = "rag_strategy.png"
download(directory, filename)

Downloaded 'rag_strategy.png' successfully.


In [11]:

#@title show image
from IPython.display import Image, display

# Specify the path to your PNG file
image_path = '/content/rag_strategy.png'

# Display the image
# Display the image with specified width and height
display(Image(filename=image_path, width=500, height=400))  # Adjust the width and height as needed

In [8]:

# Request user input for keyword parsing
user_input = input("Enter your query: ").lower()

Enter your query: what is an LLM?


## 2.3 Mean Ranking Simulation Scenario

In [9]:
#Select a score between 1 and 5 to run the simulation
ranking=5

In [10]:

# initializing the text for the generative AI model simulations
text_input=[]

## Ranking -1-2- No RAG

In [11]:
if ranking>=1 and ranking<3:
  text_input=user_input

## Ranking 3-4 : Human Expert Feedback RAG only

In [12]:
hf=False
if ranking>=3 and ranking<5:
  hf=True

In [13]:
if hf==True:
  from grequests import download
  directory = "Chapter05"
  filename = "human_feedback.txt"
  download(directory, filename)

In [14]:
if hf==True:
  # Check if 'human_feedback.txt' exists
    efile = os.path.exists('/content/human_feedback.txt')

    if efile:
        # Read and clean the file content
        with open('/content/human_feedback.txt', 'r') as file:
            content = file.read().replace('\n', ' ').replace('#', '')  # Removing new line and markdown characters
            #print(content)  # Uncomment for debugging or maintenance display
        text_input=content
        print(text_input)
    else:
      print("File not found")
      hf=False

## Ranking 5 : RAG only with no HF

In [16]:
if ranking>=5:
  max_words=100 #Limit: the size of the data we can add to the input
  rdata=process_query(user_input,max_words)
  print(rdata) # for maintenance if necessary
  if rdata:
        rdata_clean = rdata.replace('\n', ' ').replace('#', '')
        rdata_sentences = rdata_clean.split('. ')
  text_input=rdata
  print(text_input)

Fetching data from: https://en.wikipedia.org/wiki/Large_language_model

First 100 words of the cleaned text:
A large language model ( LLM ) is a type of machine learning model designed for
natural language processing tasks such as language generation . LLMs are
language models with many parameters, and are trained with self-supervised
learning on a vast amount of text. The largest and most capable LLMs are
generative pretrained transformers (GPTs). Modern models can be fine-tuned for
specific tasks or guided by prompt engineering . [ ] These models acquire
predictive power regarding syntax , semantics , and ontologies [ ] inherent in
human language corpora, but they also inherit inaccuracies and biases present in
the

Prompt for Generator: Summarize the following information about llm: A large language model ( LLM ) is
a type of machine learning model designed for natural language processing tasks
such as language generation . LLMs are language models with many parameters, and
are trai

In [15]:
print("user input:",user_input)

user input: what is an llm?


In [17]:
!pip install openai==1.55.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.4
    Uninstalling openai-1.59.4:
      Successfully uninstalled openai-1.59.4


In [20]:
#The OpenAI Key
import os
import openai
f = open("open-aikey.txt", "r")
API_KEY=f.readline()
f.close()
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [22]:
from openai import OpenAI
import time

# Initialize the OpenAI client
client = openai.OpenAI()

gpt_model = "gpt-4o-mini"
start_time = time.time()  # Start timing before the request

def call_gpt4_with_full_text(itext):
    # Join all lines to form a single string
    text_input = '\n'.join(itext)
    prompt = f"Please summarize or elaborate on the following content:\n{text_input}"

    try:
      response = client.chat.completions.create(
         model=gptmodel,
         messages=[
            {"role": "system", "content": "You are an expert Natural Language Processing exercise expert."},
            {"role": "assistant", "content": "1.You can explain read the input and answer in detail"},
            {"role": "user", "content": prompt}
         ],
         temperature=0.1  # Add the temperature parameter here and other parameters you need
        )
      return response.choices[0].message.content.strip()
    except Exception as e:
        return str(e)

# Call the function and print the result
gpt4_response = call_gpt4_with_full_text(text_input)

response_time = time.time() - start_time  # Measure response time
print(f"Response Time: {response_time:.2f} seconds")  # Print response time

print(gptmodel,"Response:", gpt4_response)

Response Time: 3.36 seconds
gpt-4o-mini Response: A large language model (LLM) is a type of machine learning model specifically designed for tasks related to natural language processing (NLP), such as language generation. These models are characterized by having a significant number of parameters and are typically trained using self-supervised learning on extensive datasets of text.

The most advanced and capable LLMs are generative pre-trained transformers (GPTs). These models can be fine-tuned for specific tasks or guided by prompt engineering to enhance their performance in particular applications.

LLMs acquire predictive capabilities concerning syntax, semantics, and ontologies that are inherent in human language. However, they also inherit inaccuracies and biases that exist within the training data. This duality highlights both the potential and the limitations of LLMs in understanding and generating human language.


In [23]:
import textwrap

def print_formatted_response(response):
    # Define the width for wrapping the text
    wrapper = textwrap.TextWrapper(width=80)  # Set to 80 columns wide, but adjust as needed
    wrapped_text = wrapper.fill(text=response)

    # Print the formatted response with a header and footer
    print("GPT-4 Response:")
    print("---------------")
    print(wrapped_text)
    print("---------------\n")

# Assuming 'gpt4_response' contains the response from the previous GPT-4 call
print_formatted_response(gpt4_response)

GPT-4 Response:
---------------
A large language model (LLM) is a type of machine learning model specifically
designed for tasks related to natural language processing (NLP), such as
language generation. These models are characterized by having a significant
number of parameters and are typically trained using self-supervised learning on
extensive datasets of text.  The most advanced and capable LLMs are generative
pre-trained transformers (GPTs). These models can be fine-tuned for specific
tasks or guided by prompt engineering to enhance their performance in particular
applications.  LLMs acquire predictive capabilities concerning syntax,
semantics, and ontologies that are inherent in human language. However, they
also inherit inaccuracies and biases that exist within the training data. This
duality highlights both the potential and the limitations of LLMs in
understanding and generating human language.
---------------



### Evaluator

#### Cosine Similarity

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])
    return similarity[0][0]

# Example usage with your existing functions
similarity_score = calculate_cosine_similarity(text_input, gpt4_response)

print(f"Cosine Similarity Score: {similarity_score:.3f}")

Cosine Similarity Score: 0.736


#### 3.3.Human user rating

In [25]:
# Score parameters
counter=20                      # number of queries
score_history=60                # human feedback
threshold=4                     # minimum rankings to trigger human expert feedback

In [26]:
import numpy as np
def evaluate_response(response):
    print("\nGenerated Response:")
    print(response)
    print("\nPlease evaluate the response based on the following criteria:")
    print("1 - Poor, 2 - Fair, 3 - Good, 4 - Very Good, 5 - Excellent")
    score = input("Enter the relevance and coherence score (1-5): ")
    try:
        score = int(score)
        if 1 <= score <= 5:
            return score
        else:
            print("Invalid score. Please enter a number between 1 and 5.")
            return evaluate_response(response)  # Recursive call if the input is invalid
    except ValueError:
        print("Invalid input. Please enter a numerical value.")
        return evaluate_response(response)  # Recursive call if the input is invalid

score = evaluate_response(gpt4_response)
print("Evaluator Score:", score)

counter+=1
score_history+=score
mean_score=round(np.mean(score_history/counter), 2)
if counter>0:
  print("Rankings      :", counter)
  print("Score history : ", mean_score)


Generated Response:
A large language model (LLM) is a type of machine learning model specifically designed for tasks related to natural language processing (NLP), such as language generation. These models are characterized by having a significant number of parameters and are typically trained using self-supervised learning on extensive datasets of text.

The most advanced and capable LLMs are generative pre-trained transformers (GPTs). These models can be fine-tuned for specific tasks or guided by prompt engineering to enhance their performance in particular applications.

LLMs acquire predictive capabilities concerning syntax, semantics, and ontologies that are inherent in human language. However, they also inherit inaccuracies and biases that exist within the training data. This duality highlights both the potential and the limitations of LLMs in understanding and generating human language.

Please evaluate the response based on the following criteria:
1 - Poor, 2 - Fair, 3 - Good, 

 ## |3.4.Human expert evaluation
Human feedback for RAG adaptive loop preparation.

Flashcard creation.

images for the expert's interface

In [27]:
counter_threshold=10
score_threshold=4

if counter>counter_threshold and score_history<=score_threshold:
  print("Human expert evaluation is required for the feedback loop.")